### This is my attempt to do shap... I hope

In [67]:
import pandas as pd
import numpy as np
import pandas_gbq
import yfinance as yf
import datetime

#### Lets get some model output data

In [36]:
project_id = 'irlts-317602'
# Full table path in gcp bigquery irlts-317602.2018.10eps_default_training
dataset_name = '2017'
table_name = '10eps_default'
r_query = """
SELECT 
    data.* 
FROM 
    `irlts-317602.2018.10eps_default_training` as data 
WHERE 
    data.Episode = '8'
"""
query_result = pandas_gbq.read_gbq(r_query,project_id=project_id, progress_bar_type=None)

In [68]:
query_result.Date = query_result.Date.astype(np.datetime64)

In [71]:
output_data = query_result.set_index('Date').sort_index()

#### Lets get some feature data

In [46]:
last_output_date = output_data.index.max()
first_output_date = output_data.index.min()
feature_df = yf.download('VOO',start=first_output_date,end=last_output_date, interval='1d')

[*********************100%***********************]  1 of 1 completed


In [73]:
feature_df_subset = feature_df[['Adj Close','Volume']]
# feature_df_subset.index

#### Lets join these two dfs (hoping this goes well)

In [89]:
final_data = query_result.join(feature_df_subset,on="Date",how = 'inner') #join the two datasets
final_data.rename(columns={f'{j}':f'{j.lower()}'for i,j in enumerate(final_data.columns)},inplace = True) #fix columns because this is bothering me
final_data.rename(columns={'Adj Close':'adj_close'},inplace=True)


,date,hold,buy,sell,choice,episode,adj close,volume
0,2018-01-26,0.879150927066803,0.7801176905632019,0.8700972199440002,0,8,243.352722,2857300
1,2018-01-29,0.8475786447525024,0.7617209553718567,0.8034783005714417,0,8,241.670990,5145000
2,2018-02-01,0.6819868683815002,0.6323200464248657,0.6294571757316589,0,8,239.074539,3125700
3,2018-02-26,2.5456316471099854,2.146250009536743,2.346653461456299,0,8,235.997467,1892000
4,2018-02-27,2.6943464279174805,2.3414077758789062,2.477212429046631,0,8,232.985107,2947100
...,...,...,...,...,...,...,...,...
193,2018-10-01,0.5459641218185425,0.8462108373641968,1.290140986442566,2,8,251.103226,1968800
194,2018-10-02,0.4659746587276459,0.7753522992134094,1.2290778160095215,2,8,251.075134,3034100
195,2018-10-05,0.3356887996196747,0.5928556323051453,0.9725805521011353,2,8,247.824387,2564700
196,2018-10-08,0.29206451773643494,0.5181288719177246,0.857489824295044,2,8,247.758804,3219600
